In [4]:
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import datetime
import pyspark.sql.functions as func
from pyspark.sql.functions import col
os.environ["PYSPARK_PYTHON"]="/home/hadoop/anaconda3/bin/python"

In [ ]:
def toDate(inputStr):
    newStr = ""
    if len(inputStr) == 8:
        s1 = inputStr[0:4]
        s2 = inputStr[5:6]
        s3 = inputStr[7]
        newStr = s1 + "-" + "0" + s2 + "-" + "0" + s3
    elif len(inputStr) == 9:
        s1 = inputStr[0:4]
        if inputStr[6] == '/':
            s2 = inputStr[5:6]
            s3 = inputStr[7:9]
            newStr = s1 + "-" + "0" + s2 + "-" + s3
        else:
            s2 = inputStr[5:7]
            s3 = inputStr[8]
            newStr = s1 + "-" + s2 + "-" + "0" + s3
    else:
        s1 = inputStr[0:4]
        s2 = inputStr[5:7]
        s3 = inputStr[8:10]
        newStr = s1 + "-" + s2 + "-" + s3
    date = datetime.strptime(newStr, "%Y-%m-%d")
    return date

def getDstCoarse(inputStr):
    inputStr = inputStr.lstrip()
    newdst = ""
    if inputStr == '海澜之家':
        newdst = inputStr
    else:
        if len(inputStr) == 4:
            newdst = inputStr[0]
        elif len(inputStr) == 5:
            newdst = inputStr[0]
        else:
            newdst = 'other'  
    return newdst
def getDstFine(inputStr):
    inputStr = inputStr.lstrip()
    newdst = ""
    if inputStr == '海澜之家':
        newdst = inputStr
    else:
        if len(inputStr) == 4:
            newdst = inputStr[0]
        elif len(inputStr) == 5:
            newdst = inputStr[0] + inputStr[1]
        else:
            newdst = 'other'    
    return newdst

        
spark = SparkSession.builder.config(conf = SparkConf()).getOrCreate()
fields = [StructField("date", DateType(), False),\
          StructField("date_order", IntegerType(), False),\
          StructField("dst", StringType(), True),\
          StructField("dst_type", IntegerType(), False),\
          StructField("status", IntegerType(), False),
         ]
schema = StructType(fields)
rdd0 = spark.sparkContext.textFile("/user/hadoop/2019.txt")
rdd1 = rdd0.map(lambda x : x.split("\t")).map(lambda p : Row(toDate(p[0]), int(p[1]), getDstCoarse(p[2]), int(p[3]), int(p[4])))

shemaNAInfo = spark.createDataFrame(rdd1, schema)
shemaNAInfo.createOrReplaceTempView("naInfo")

# 1.计算每日的报修单数
df = shemaNAInfo.filter(col("status") == 5).groupBy("date").agg(func.count("date_order")).sort(shemaNAInfo["date"].asc())

# 列重命名
df1 = df.withColumnRenamed("count(date_order)", "date_order")
df1.repartition(1).write.json("NA1.json") # 写入hdfs

# 注册为临时表
df1.createOrReplaceTempView("ustotal")

# 2.计算每栋楼报修数
df2 = spark.sql("select dst, count(dst) as cdst from naInfo where status = 5 group by dst")
df2.sort(df2["dst"].asc()).repartition(1).write.json("NA2.json")  # 写入hdfs

# 3.男女报修数

df3 = spark.sql("select dst_type, count(dst_type) as sexcount from naInfo where status = 5 group by dst_type")
df3.sort(df3["dst_type"].asc()).repartition(1).write.json("NA3.json") # 写入hdfs

# 4.每月报修数
df4 = spark.sql("select MONTH(date) as month, count(MONTH(date)) as count from naInfo where status = 5 group by month")
df4.sort(df4["month"].asc()).repartition(1).write.json("NA4.json") # 写入hdfs